In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os
import json

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

substance_selection_widget = widgets.MoleculeSelectionWidget()
substance_selection_widget.load_dropdown_box()

instrument_selector = widgets.InstrumentSelectionWidget()
instrument_selector.load_dropdown_box()

srd_widgets = widgets.ObjectPropertiesWidgets("Sublimation Rate Determination")
srd_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def create_srd_openbis(b):
    object_properties = {}
    for prop in CONFIG["objects"]["SRD"]["properties"]:
        if prop == "evaporator_slot":
            object_properties[prop] = json.dumps({"evaporator_number": srd_widgets.properties_widgets[prop].children[0].value, 
                                                    "details": srd_widgets.properties_widgets[prop].children[1].value})
        elif CONFIG["properties"][prop]["property_type"] == "VARCHAR":
            object_properties[prop] = srd_widgets.properties_widgets[prop].value
        elif CONFIG["properties"][prop]["property_type"] == "QUANTITY_VALUE":
            object_properties[prop] = json.dumps({"has_value": srd_widgets.properties_widgets[prop].children[0].value, 
                                                    "has_unit": srd_widgets.properties_widgets[prop].children[1].value})
        
    srd_parents = [substance_selection_widget.dropdown_boxes.children[0].value, instrument_selector.dropdown_boxes.children[0].value]

    results_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="SUBLIMATION_RATE_DETERMINATION", 
        collection = "/LOG_ENTRIES/LOG_ENTRIES/SRD_COLLECTION", 
        props = object_properties, parents = srd_parents
    )
    upload_datasets(results_object, srd_support_files)
    
    display(utils.Javascript(data = "alert('Upload successful!')"))

def upload_datasets(method_object, support_files_widget):
    for filename in support_files_widget.value:
        file_info = support_files_widget.value[filename]
        utils.save_file(file_info['content'], filename)
        OPENBIS_SESSION.new_dataset(type = 'RAW_DATA', sample = method_object, files = [filename]).save()
        os.remove(filename)

# Create sublimation rate determination object

## Select molecule

In [ ]:
display(substance_selection_widget)

## Select instrument

In [ ]:
display(instrument_selector)

## Properties

In [ ]:
display(srd_widgets)

## Support files

In [ ]:
display(srd_support_files)

## Save

In [ ]:
substance_selection_widget.dropdown_boxes.children[0].observe(substance_selection_widget.load_metadata, names = 'value')
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_srd_openbis)
quit_button.on_click(close_notebook)